In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from jcopml.pipeline import num_pipe, cat_pipe
from jcopml.utils import save_model, load_model
from jcopml.plot import plot_missing_value
from jcopml.feature_importance import mean_score_decrease

In [6]:
df = pd.read_csv("../../Data/wa/Data gede masked olahan.csv", parse_dates=['Date','Last Update Time'])
df.head()

C:\Users\Jayy\AppData\Local\Temp\ipykernel_10512\460260287.py:1: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../../Data/wa/Data gede masked olahan.csv", parse_dates=['Date','Last Update Time'])


,Email User,Nomor Telepon Tujuan,Status,Date,Last Update Time,jam last update,tanggal last update,hari last update,isWeekend last update,jam,tanggal,hari,Nomor Telepon Tujuan Prefix,provider,selisih
0,Email user 1,0821xxxxxxxx,read,2023-02-14 11:13:00,2023-02-14 11:14:00,11,14,Selasa,0,11,14,Selasa,0821,Telkomsel,0 days 00:01:00
1,Email user 1,0812xxxxxxxx,read,2023-02-14 11:13:00,2023-02-14 11:17:00,11,14,Selasa,0,11,14,Selasa,0812,Telkomsel,0 days 00:04:00
2,Email user 1,0838xxxxxxxx,read,2023-02-14 11:13:00,2023-02-14 11:15:00,11,14,Selasa,0,11,14,Selasa,0838,XL,0 days 00:02:00
3,Email user 1,0858xxxxxxxx,read,2023-02-14 11:13:00,2023-02-14 11:28:00,11,14,Selasa,0,11,14,Selasa,0858,Indosat,0 days 00:15:00
4,Email user 1,0858xxxxxxxx,delivered,2023-02-14 11:13:00,2023-02-14 11:13:00,11,14,Selasa,0,11,14,Selasa,0858,Indosat,0 days 00:00:00


In [7]:
groupby = df.groupby(['Email User','hari last update', 'jam last update', 'Status']).size().unstack().fillna(0)
groupby.reset_index(inplace=True)

In [8]:
groupby

Status,Email User,hari last update,jam last update,delivered,failed,in_queue,read,rejected,submitted
0,Email user 1,Jumat,0,0.0,0.0,0.0,2.0,0.0,0.0
1,Email user 1,Jumat,1,0.0,0.0,0.0,2.0,0.0,0.0
2,Email user 1,Jumat,2,0.0,0.0,0.0,1.0,0.0,0.0
3,Email user 1,Jumat,5,0.0,0.0,0.0,1.0,0.0,0.0
4,Email user 1,Jumat,7,1.0,0.0,0.0,3.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
23334,Email user 99,Rabu,12,0.0,0.0,0.0,0.0,0.0,42449.0
23335,Email user 99,Rabu,13,0.0,2.0,0.0,0.0,0.0,18008.0
23336,Email user 99,Senin,13,0.0,1.0,0.0,0.0,0.0,0.0
23337,Email user 99,Senin,14,0.0,8.0,0.0,0.0,0.0,0.0


In [9]:
from sklearn.preprocessing import RobustScaler, MinMaxScaler

scaller = MinMaxScaler()
groupby['read scalled'] = scaller.fit_transform(groupby[['read']])
groupby['delivered scalled'] = scaller.fit_transform(groupby[['delivered']])
groupby['test'] = (groupby['delivered scalled'] * 0.1) + (groupby['read scalled'] * 0.9)
groupby

Status,Email User,hari last update,jam last update,delivered,failed,in_queue,read,rejected,submitted,read scalled,delivered scalled,test
0,Email user 1,Jumat,0,0.0,0.0,0.0,2.0,0.0,0.0,0.000026,0.000000,0.000023
1,Email user 1,Jumat,1,0.0,0.0,0.0,2.0,0.0,0.0,0.000026,0.000000,0.000023
2,Email user 1,Jumat,2,0.0,0.0,0.0,1.0,0.0,0.0,0.000013,0.000000,0.000012
3,Email user 1,Jumat,5,0.0,0.0,0.0,1.0,0.0,0.0,0.000013,0.000000,0.000012
4,Email user 1,Jumat,7,1.0,0.0,0.0,3.0,0.0,0.0,0.000039,0.000025,0.000037
...,...,...,...,...,...,...,...,...,...,...,...,...
23334,Email user 99,Rabu,12,0.0,0.0,0.0,0.0,0.0,42449.0,0.000000,0.000000,0.000000
23335,Email user 99,Rabu,13,0.0,2.0,0.0,0.0,0.0,18008.0,0.000000,0.000000,0.000000
23336,Email user 99,Senin,13,0.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
23337,Email user 99,Senin,14,0.0,8.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000


In [10]:
groupby[(groupby['hari last update'] == 'Senin')].sort_values(by=['test'], ascending=False).head(5)

Status,Email User,hari last update,jam last update,delivered,failed,in_queue,read,rejected,submitted,read scalled,delivered scalled,test
8194,Email user 161,Senin,13,13565.0,6817.0,0.0,22617.0,0.0,3336.0,0.292194,0.341430,0.297118
8196,Email user 161,Senin,15,8247.0,3718.0,0.0,21988.0,0.0,2196.0,0.284068,0.207576,0.276419
8197,Email user 161,Senin,16,5197.0,1974.0,0.0,20199.0,1.0,1051.0,0.260956,0.130808,0.247941
8195,Email user 161,Senin,14,4128.0,1580.0,0.0,19173.0,0.0,738.0,0.247700,0.103901,0.233320
16150,Email user 39,Senin,12,4009.0,0.0,0.0,15712.0,6448.0,1054.0,0.202987,0.100906,0.192779
